In [2]:
import pandas as pd
import numpy as np

In [190]:
class CarData:
    
    import pandas as pd
    
    missing = 0
    
    def __init__(self, path):
        
        #read all .csv files from the directory
        self.data = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, "*.csv"))), sort=False)
        self.data.drop_duplicates()

        #rename columns
        self.data.columns = ['title', 'price', 'sub_title', 'mileage_km', 'year', 'engine_cm3',
                'engine_type', 'city', 'province', 'negotiable']
        
        #drop NaNs and save the number of rows dropped to the missing varaible
        self.missing = self.data['engine_type'].isna().sum()
        self.data.dropna(subset = ['engine_type'], axis = 'index', inplace = True)
        
        self.missing = self.missing + self.data['city'].isna().sum()
        self.data.dropna(subset = ['city'], axis = 'index', inplace = True)
        
        #clean up the columns
        self.data['price'] = self.data['price'].apply(lambda x: x.replace(" ", "")).astype('int')
        
        self.data['mileage_km'] = self.data['mileage_km'].apply(lambda x: x.replace("km", ""))
        self.data['mileage_km'] = self.data['mileage_km'].apply(lambda x: x.replace(" ", "")).astype('int')
        
        self.data['engine_cm3'] = self.data['engine_cm3'].astype('str')
        self.data['engine_cm3'] = self.data['engine_cm3'].apply(lambda x: x.replace('cm3', ''))
        self.data['engine_cm3'] = self.data['engine_cm3'].apply(lambda x: x.replace(' ','')).astype('int')
        
        self.data['province'] = self.data['province'].astype('str')
        self.data['province'] = self.data['province'].apply(lambda x: x.replace('(',''))
        self.data['province'] = self.data['province'].apply(lambda x: x.replace(')',''))

        

In [191]:
x = CarData('data/')

In [192]:
x.missing

418

In [193]:
x.data.head()

,title,price,sub_title,mileage_km,year,engine_cm3,engine_type,city,province,negotiable
0,Smart Fortwo cabrio,18800,Cabrio Automat Nawigacja Kamera Led Szwajcaria,37000,2016,999,Benzyna,Bielsko-Biała,Śląskie,"Do negocjacji, Faktura VAT"
1,Ford Kuga,34700,Nawigacja/kamera cofania/Zadbany/,198000,2011,2000,Diesel,Dąbrowa,Pomorskie,Do negocjacji
2,Chevrolet Captiva 2.4,27900,2.4 136km klima alus navi DVD skóra 4x4 Poleca...,246000,2009,2405,Benzyna,Płock,Mazowieckie,Do negocjacji
3,Renault Clio 1.2,37900,1.2 BENZ 73 KM !! IDEALNY !! FULL !! Warszawa,15000,2017,1149,Benzyna,Warszawa,Mazowieckie,Do negocjacji
4,Opel Meriva,7900,mały przebieg 109 tyś/km bezwypadkowy,109000,2006,1400,Benzyna,Żyrardów,Mazowieckie,NaN


In [170]:
x.data

,title,price,sub_title,mileage_km,year,engine_cm3,engine_type,city,province,negotiable
0,Smart Fortwo cabrio,18800,Cabrio Automat Nawigacja Kamera Led Szwajcaria,37000,2016,999,Benzyna,Bielsko-Biała,Śląskie,"Do negocjacji, Faktura VAT"
1,Ford Kuga,34700,Nawigacja/kamera cofania/Zadbany/,198000,2011,2000,Diesel,Dąbrowa,Pomorskie,Do negocjacji
2,Chevrolet Captiva 2.4,27900,2.4 136km klima alus navi DVD skóra 4x4 Poleca...,246000,2009,2405,Benzyna,Płock,Mazowieckie,Do negocjacji
3,Renault Clio 1.2,37900,1.2 BENZ 73 KM !! IDEALNY !! FULL !! Warszawa,15000,2017,1149,Benzyna,Warszawa,Mazowieckie,Do negocjacji
4,Opel Meriva,7900,mały przebieg 109 tyś/km bezwypadkowy,109000,2006,1400,Benzyna,Żyrardów,Mazowieckie,NaN
...,...,...,...,...,...,...,...,...,...,...
4987,Renault Clio 1.2,28900,Nawigacja Klimatyzacja Esp Stan Wzorowy Gwaran...,124000,2014,1149,Benzyna,Imielin,Śląskie,Do negocjacji
4988,Kia Ceed 1.4,22500,"benzyna , czujniki, tempomat",190566,2011,1396,Benzyna,Rawa Mazowiecka,Łódzkie,Do negocjacji
4989,Ford Focus,10900,NaN,226000,2016,1596,Benzyna+LPG,Złotów,Wielkopolskie,NaN
4990,Audi A6,55000,"Ładne a6,2014r,duża nawigacja,skóry,4xclimatronik",185000,2014,1984,Benzyna,Warszawa,Mazowieckie,NaN


,title,price,sub_title,mileage_km,year,engine_cm3,engine_type,city,province,negotiable
13,Honda CR-V,22500,NaN,190000,2007,Diesel,NaN,Brzostek,Podkarpackie,Do negocjacji
123,Tesla Model S 90,299997,europejska specyfikacja Autopilot supercharger...,79000,2017,Elektryczny,NaN,Zawiercie,Śląskie,"Możliwość odliczenia VAT, Faktura VAT"
157,Ford KA,13999,Super stan Full opcja klimatronik,130000,2010,Benzyna,NaN,Legnica,Dolnośląskie,NaN
158,BMW Seria 3,119000,M Pakiet,98888,2017,Diesel,NaN,Czermin,Wielkopolskie,NaN
160,Ford KA,13999,Super stan Full opcja klimatronik,130000,2010,Benzyna,NaN,Legnica,Dolnośląskie,NaN
...,...,...,...,...,...,...,...,...,...,...
4926,Volkswagen Golf,89000,"Comfortline, 2.0 TDI BMT 150KM, manualna, NAWI...",5,2019,Diesel,NaN,Poznań,Wielkopolskie,Faktura VAT
4938,Renault Zoe,96900,"Life, FV-23%, gwarancja",9384,2018,Elektryczny,NaN,Jawornik,Małopolskie,Faktura VAT
4952,Renault Megane,18500,Xenon Panoramadach Navi,160125,2010,Diesel,NaN,Czarnków,Wielkopolskie,NaN
4953,Renault Zoe,96900,"Life, FV-23%, gwarancja",11466,2018,Elektryczny,NaN,Jawornik,Małopolskie,Faktura VAT


In [161]:
x.data.isnull().sum()

title             0
price             0
sub_title      1159
mileage_km        0
year              0
engine_cm3        0
engine_type     418
city              1
province          0
negotiable     5306
dtype: int64

In [15]:
data = pd.read_csv('run_results_21-04-2020.csv')

In [38]:
data = ReadAndPreprocess(df)

AttributeError: 'float' object has no attribute 'replace'

In [52]:
df = pd.read_csv('data/run_results_21-04-2020.csv')
df.head()

,title_name,title_price,title_sub_title,title_mileage,title_year,title_engine_l,title_engine_type,title_localization_1,title_localization_2,title_negotiable
0,Volkswagen Golf,14 900,Sprzedam 1.9 ARL TURBO MOC 300 KM . Zarejestro...,160 000 km,2008,1 900 cm3,Diesel,Pęzino,(Zachodniopomorskie),Do negocjacji
1,BMW Seria 4 420d,99 000,BMW Seria 4,88 000 km,2015,1 995 cm3,Diesel,Pszczyna,(Śląskie),NaN
2,Audi A4,28 900,1.8 T tylko 102000 tyś. km. przebiegu grzane f...,102 000 km,2009,1 800 cm3,Benzyna,Góra,(Dolnośląskie),Do negocjacji
3,Opel Zafira,54 900,1.4 Turbo 120 KM ! Klimatronic ! Naviagacja !,88 500 km,2016,1 364 cm3,Benzyna,Baranów,(Lubelskie),NaN
4,Škoda Fabia 1.4,6 900,"1,4 benzyna UWAGA niski przebieg",75 000 km,2001,1 397 cm3,Benzyna,Pułtusk,(Mazowieckie),Do negocjacji


In [53]:
df2 = pd.read_csv('data/run_results_25-04_2020.csv')
df2.head()

,title_name,title_price,title_sub_title,title_mileage,title_year,title_engine_l,title_engine_type,title_localization_1,title_localization_2,title_negotiable
0,Renault Captur,52 700,Gwarancja-Full Wersja-Black Deasing-Navi-Kamer...,15 500 km,2019,1 200 cm3,Benzyna,Gniezno,(Wielkopolskie),Do negocjacji
1,Renault Trafic,24 900,Vivaro 2.5 DCi 135KM 23000km Klimatyzacja Tył ...,230 000 km,2005,2 463 cm3,Diesel,Jarocin,(Wielkopolskie),NaN
2,Audi A4 Cabriolet,8 000,Audi A4B6 3.0 Bez+Gaz Sprzedam/Zamienie,235 000 km,2004,2 976 cm3,Benzyna+LPG,Tczew,(Pomorskie),Do negocjacji
3,BMW Seria 5,45 000,opis samochodu w zdjęciach,304 000 km,2011,Diesel,NaN,Psary,(Śląskie),Do negocjacji
4,Seat Altea,14 900,"2005rok 1,9TDI 105KM",171 505 km,2005,1 896 cm3,Diesel,Kalisz,(Wielkopolskie),Do negocjacji


In [54]:
df3 = pd.read_csv('data/run_results_26-04-2020.csv')
df3.head()

,title_name,title_price,title_mileage,title_year,title_engine_l,title_engine_type,title_localization_1,title_localization_2,title_negotiable,title_sub_title
0,Volkswagen Beetle 2.0,42 900,64 000 km,2012,1 984 cm3,Benzyna,Borkowo Kościelne,(Mazowieckie),Do negocjacji,NaN
1,Škoda Fabia 1.2,6 990,188 000 km,2006,1 198 cm3,Benzyna,Niegosław,(Lubuskie),Do negocjacji,benzyna 1.2 / klima / lift
2,Mazda 3 1.6,8 000,213 169 km,2005,1 598 cm3,Benzyna,Warszawa,(Mazowieckie),Do negocjacji,Mazda 3 I Exclusive 2005 1.6 benzyna 105KM
3,Audi A6 3.0,21 900,300 000 km,2005,2 967 cm3,Diesel,Kamień Pomorski,(Zachodniopomorskie),Do negocjacji,3.0 TDI S LINE Bose Super Stan Quattro Szyberdach
4,Opel Astra,9 500,216 000 km,2004,1 364 cm3,Benzyna,Iłowa,(Lubuskie),Do negocjacji,*H*1.4*Benzyna*Klimatyzacja*Bezwypadkowy*Orygi...


In [84]:
x.df_head()